<a href="https://colab.research.google.com/github/nomura-st/gas_questionnaire/blob/main/%E5%85%AC%E9%96%8B%E6%98%A0%E7%94%BB%E3%82%B9%E3%82%B1%E3%82%B8%E3%83%A5%E3%83%BC%E3%83%AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
#############################
### カスタム情報
#############################
# 読み書き対象スプレッドシートID
ssID='xxxxxxs'
# 読み書き対象スプレッドシート シート名
sheetName='上映中'
# データ表が始まる行
rowStart=2

# 検索対象サイト
baseURL = 'https://eiga.com'
# 対象の県
prefURL = f'{baseURL}/theater/10'
paths = {
    "ユナイテッド・シネマ 前橋": "/100101/3229/",
    "109シネマズ高崎": "/100201/3230/",
    "イオンシネマ高崎": "/100201/3231/",
}


In [26]:
import requests
import re

from bs4 import BeautifulSoup  # BeautifulSoupクラスをインポート

def select(url, selector):
  html = requests.get(url)
  # 取得したHTMLをBeautifulSoupを使ってパースします。
  soup = BeautifulSoup(html.content, "html.parser")
  # パースしたHTMLから特定の要素を抽出します。結果はリストで返ってきます。
  elem = soup.select(selector)

  return elem

In [27]:
def getMoviesFromEiga(url, path):
  # パースしたHTMLから特定の要素を抽出します。結果はリストで返ってきます。
  movies = select(url + path, "main .content-container section[data-title]")
  obj = list(map(lambda movie: {
      "name": movie["data-title"],
      "schedules": list(map(lambda t: {
          "type": t.select(".movie-type")[0].get_text("/") if len(t.select(".movie-type")) > 0 else "",
          "date": list(map(lambda d: d["data-time"], t.select("td [data-time]"))),
      }, movie.select(".movie-schedule"))),
      "image": movie.select(".movie-image img")[0]["src"] if len(movie.select(".movie-image img")) > 0 else None,
      "time": list(filter(lambda s: s.get_text().endswith("分"), movie.select(".movie-image .data span")))[0].get_text().replace("分", ""),
      "link": baseURL + movie.select("h2 a")[0]["href"],
    }, movies))
  
  return obj


In [28]:
#import json

theaters = list(map(lambda name: {
    "name": name,
    "movies": getMoviesFromEiga(prefURL, paths[name]),
}, paths.keys()))

# print (json.dumps(theaters, ensure_ascii=False))

In [29]:
# 認証のためのコード
from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

In [33]:
import datetime

ss = gc.open_by_key(ssID)
sheet = ss.worksheet(sheetName)

# 既存を削除
ss.values_clear(f'\'{sheetName}\'!B2:J501')

row = rowStart
# 日時１つを１行として書き込み
for theater in theaters:
  for movie in theater["movies"]:
    for schedule in movie["schedules"]:
      for d in schedule["date"]:
        dt = datetime.datetime.fromtimestamp(int(d), datetime.timezone.utc)
        # 日本のタイムゾーン調整
        dt = dt  + datetime.timedelta(hours = 9)
        # 終了時刻は目安 ＋１０分
        dtEnd = dt + datetime.timedelta(minutes = int(movie["time"]) + 10)
        dateStr = dt.strftime('%Y/%m/%d')
        timeStartStr = dt.strftime('%H:%M:%S')
        timeStartStrAbout = dt.strftime('%H時台(%p)')
        timeEndStr = dtEnd.strftime('%H:%M:%S')

        # 書き込みデータ
        ds= sheet.range(f'B{row}:J{row}')
        ds[0].value = movie["name"]
        ds[1].value = schedule["type"]
        ds[2].value = dateStr
        ds[3].value = timeStartStrAbout
        ds[4].value = timeStartStr
        ds[5].value = timeEndStr
        ds[6].value = movie["time"]
        ds[7].value = theater["name"]
        ds[8].value = movie["link"]
        # 書き込み
        sheet.update_cells(ds)

        row = row + 1
